# Test for shapefile interactions

Small test to load shapefiles and plot them using geopandas and matplotlib. Also to get values form a xarray dataset using the shapefile polygon boundaries.

In [ ]:
# libraries
import xarray as xr
import geopandas as gpd
import rasterio.features
import numpy as np

# plotting
import matplotlib.pyplot as plt

## Basic shapefile interactions

Some tests to interact with shapefiles using geopandas

In [ ]:
# load the shapefile which has many polygons. Explicitely use utf8 encoding
SiteArea = gpd.read_file('data/AreaSites/AreaSites.shp', encoding='utf-8')

fig, ax = plt.subplots(1, 2, figsize=(12, 6))  # Creates a figure with two subplots

# plot all polygons
SiteArea.plot(ax=ax[0])
ax[0].set_title('Site Area')

# plot first geometry in file
SiteArea.loc[[0]].geometry.plot(ax=ax[1])
ax[1].set_title('First Site only')

In [ ]:
# getting polygon data by index
site1 = SiteArea.loc[[4]]
# here using total_bounds, not bounds, as it returns a numpy array
bounds = site1.total_bounds
min_x, min_y, max_x, max_y = bounds
print('x: ', min_x, max_x)
print('y: ', min_y, max_y)

In [ ]:
# Also, to be sure about the CRS it is important to read it
print(SiteArea.crs)

## Indexing all values inside a shapefile polygon

this is an important step to get all values of a specific testing site.

In [ ]:
# load zarr file with raster data
deadwood = xr.open_zarr('data/MD.zarr')

In [ ]:
# select a site number which might get into a loop later
Sitenumber = 2
fig, ax = plt.subplots(1, 2, figsize=(12, 6))  # Creates a figure with two subplots

# Create a subset of the zarr data with the bounds of the site
site = SiteArea.loc[[Sitenumber]]
bounds = site.total_bounds
min_x, min_y, max_x, max_y = bounds
deadwood_subset = deadwood.sel(x=slice(min_x, max_x), y=slice(min_y, max_y))

deadwood_subset.deadwood.isel(time=0).plot(ax=ax[0], add_colorbar=False)
ax[0].set_title('Deadwood in Site Area bounds')

# get data from shape with rasterio.features.geometry_mask.
# This is a numpy mask for the deadwood subset
mask = rasterio.features.geometry_mask(
    geometries=site.geometry,
    out_shape=(deadwood_subset.x.size, deadwood_subset.y.size),
    transform=deadwood_subset.rio.transform(),
    invert=True
)

# apply the mask to the deadwood_subset data
deadwood_masked = deadwood_subset.where(mask, np.nan)

# plot the masked data for a time step
deadwood_masked.deadwood.isel(time=0).plot(ax=ax[1], add_colorbar=False)
ax[1].set_title('Deadwood in Site Area shape')

# Get cubo center/edges

Last part is important for cubo. You need to find the center and then the surrounding box which covers the Site area. Here it is important to find it according to the Geographic CRS.

In [ ]:
#Find centroid in lat/lon in WGS84 (EPSG:4326)
centroid = site.centroid
centroid = centroid.to_crs(epsg=4326).iloc[0]

convert the Site to UTM and calculate the maximum edge size 

In [ ]:
# First, find out the UTM zone for the centroid and set CRS to the appropriate UTM zone of the center point
utm_zone = int((centroid.x + 180) // 6) + 1
utm_crs = f'epsg:326{utm_zone}' if centroid.y >= 0 else f'epsg:327{utm_zone}'

# Project the GeoDataFrame to UTM CRS
site_utm = site.to_crs(utm_crs)

# Get the bounding box in the UTM CRS
bounds_utm = site_utm.geometry.total_bounds
min_x, min_y, max_x, max_y = bounds_utm

# Calculate width and height of the bounding box
width = max_x - min_x
height = max_y - min_y
edge_size = max(width, height)

print(f"edge_size: {edge_size} meters")